In [1]:
import json
import numpy as np
import time
from gridfs import GridFS
from pymongo import MongoClient
from PIL import Image

from padma.models.vazios.vazio2 import VazioSVMModel

from ajna_commons.flask.conf import DATABASE, MONGODB_URI
from ajna_commons.utils.images import generate_batch, get_imagens_recortadas

db = MongoClient(host=MONGODB_URI)[DATABASE]
fs = GridFS(db)

model = VazioSVMModel()
print(model.input_shape)

2019-03-28 11:00:17,109 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  /home/ivan/pybr/ajna/padma/ajna_commons/flask/error.log
Fazendo log de atividade no arquivo  /home/ivan/pybr/ajna/padma/ajna_commons/flask/access.log
[16384]


In [12]:
batch_size = 16
limit = 32
TEMBBOX = {'metadata.contentType': 'image/jpeg',
          'metadata.predictions.bbox': {'$exists': True}}
dict_filtro = TEMBBOX
projection = ['metadata.predictions']
dict_filtro['metadata.predictions.vazio'] = {'$exists': True}

In [13]:
db['fs.files'].count_documents(dict_filtro)

33879

In [14]:
batch_gen = generate_batch(db, filtro=dict_filtro, projection=projection,
                           batch_size=batch_size, limit=limit)

model.model.n_jobs = 1

In [15]:
X = np.zeros((batch_size, *model.input_shape), dtype=np.float32)
_ids = []
y = []
s = time.time()
for batch, rows in batch_gen:
    if len(batch) == 0:
        break
    s0 = time.time()
    for i, (images, row) in enumerate(zip(batch, rows)):
        x = model.prepara(images[0])
        X[i, :] = x
        _ids.append(row['_id'])
    s1 = time.time()
    print('Montou X em %0.2f ' % (s1 - s0))
    preds = model.model.predict(X)
    y.append(preds)
    s2 = time.time()
    print('Fez predição em %s' % (s2 - s1))
    # print(indexes)
s3 = time.time()
elapsed = s3 - s
print('Tempo total: %s. Por imagem: %s' % (elapsed, elapsed / limit))
    

Montou X em 0.01 
Fez predição em 0.6319046020507812
Montou X em 0.01 
Fez predição em 0.63651442527771
Montou X em 0.01 
Fez predição em 0.6336231231689453
Montou X em 0.02 
Fez predição em 0.6324799060821533
Montou X em 0.02 
Fez predição em 0.7052021026611328
Montou X em 0.01 
Fez predição em 0.6695122718811035
Montou X em 0.02 
Fez predição em 0.6741838455200195
Montou X em 0.02 
Fez predição em 0.6316237449645996
Montou X em 0.02 
Fez predição em 0.6346638202667236
Montou X em 0.02 
Fez predição em 0.6303274631500244
Tempo total: 8.269472122192383. Por imagem: 0.05168420076370239


## Abaixo, testes e visualizações de "sanidade", isto é, as predições são iguais às do PADMA???

In [16]:
preds.shape

(16,)

In [17]:
ystack = np.vstack(y).astype(np.float32).flatten()
ystack.shape

(160,)

In [18]:
len(_ids)

160

In [19]:
# TODO: Fazer Update no MongoDB

In [27]:
for i, _id in enumerate(_ids[:10]):
    index_row = db.fs.files.find_one({'_id': _id})
    old_vazio = index_row['metadata']['predictions'][0]['vazio']
    new_vazio = ystack[i]
    print(old_vazio, new_vazio, new_vazio < 0.5, int(new_vazio) == 1)
    # print(index_row['metadata']['predictions'])

False 1.0 False True
True 0.0 True False
False 1.0 False True
False 1.0 False True
False 1.0 False True
False 1.0 False True
False 1.0 False True
False 1.0 False True
False 1.0 False True
False 1.0 False True
